In [1]:
import pandas as pd
import numpy as np

/tmp/ipykernel_2935/2162656668.py:1: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


In [2]:
df_ratings = pd.read_csv('ratings_train.txt', sep='\t')
df_ratings[:3]

,id,document,label
0,9976970,아 더빙.. 진짜 짜증나네요 목소리,0
1,3819312,흠...포스터보고 초딩영화줄....오버연기조차 가볍지 않구나,1
2,10265843,너무재밓었다그래서보는것을추천한다,0


In [3]:
df_ratings['label'].unique()

array([0, 1])

In [4]:
# 숫자 삭제
df_ratings['document_preprocess'] = df_ratings['document'].str.replace('[0-9]','',regex=True)

In [5]:
# 영문자 소문자로 통일
df_ratings['document_preprocess'] = df_ratings['document_preprocess'].str.lower()

In [6]:
# 결측치 제거
df_ratings.dropna()

,id,document,label,document_preprocess
0,9976970,아 더빙.. 진짜 짜증나네요 목소리,0,아 더빙.. 진짜 짜증나네요 목소리
1,3819312,흠...포스터보고 초딩영화줄....오버연기조차 가볍지 않구나,1,흠...포스터보고 초딩영화줄....오버연기조차 가볍지 않구나
2,10265843,너무재밓었다그래서보는것을추천한다,0,너무재밓었다그래서보는것을추천한다
3,9045019,교도소 이야기구먼 ..솔직히 재미는 없다..평점 조정,0,교도소 이야기구먼 ..솔직히 재미는 없다..평점 조정
4,6483659,사이몬페그의 익살스런 연기가 돋보였던 영화!스파이더맨에서 늙어보이기만 했던 커스틴 ...,1,사이몬페그의 익살스런 연기가 돋보였던 영화!스파이더맨에서 늙어보이기만 했던 커스틴 ...
...,...,...,...,...
149995,6222902,인간이 문제지.. 소는 뭔죄인가..,0,인간이 문제지.. 소는 뭔죄인가..
149996,8549745,평점이 너무 낮아서...,1,평점이 너무 낮아서...
149997,9311800,이게 뭐요? 한국인은 거들먹거리고 필리핀 혼혈은 착하다?,0,이게 뭐요? 한국인은 거들먹거리고 필리핀 혼혈은 착하다?
149998,2376369,청춘 영화의 최고봉.방황과 우울했던 날들의 자화상,1,청춘 영화의 최고봉.방황과 우울했던 날들의 자화상


In [7]:
!pip install python-mecab-ko

In [8]:
from mecab import MeCab
mecab = MeCab()

In [9]:
mecab.pos(df_ratings['document_preprocess'][1])

[('흠', 'IC'),
 ('.', 'SF'),
 ('..', 'SY'),
 ('포스터', 'NNP'),
 ('보고', 'JKB'),
 ('초딩', 'NNG'),
 ('영화', 'NNG'),
 ('줄', 'NNG'),
 ('.', 'SF'),
 ('...', 'SY'),
 ('오버', 'NNG'),
 ('연기', 'NNG'),
 ('조차', 'JX'),
 ('가볍', 'VA'),
 ('지', 'EC'),
 ('않', 'VX'),
 ('구나', 'EC')]

In [10]:
df_ratings['document_preprocess'] = df_ratings['document_preprocess'].astype(str)

In [11]:
def mecab_tag(text):
    morphs_nouns = []
    morphs_nouns_list = []
    for sentence in text:
        # 명사, 형용사, 동사
        morphs = [morph for morph, tag in mecab.pos(sentence) if tag in ['NNG', 'NNP', 'NNB', 'NNBC', 'NP', 'NR', 'VA', 'VV']]
        morphs_nouns.append(' '.join(morphs))
        # morphs_nouns_list = morphs_nouns_list + morphs
    return morphs_nouns

In [12]:
morphs_nouns = mecab_tag(df_ratings['document_preprocess'])

In [13]:
df_ratings['document_clean'] = morphs_nouns

In [14]:
df_ratings

,id,document,label,document_preprocess,document_clean
0,9976970,아 더빙.. 진짜 짜증나네요 목소리,0,아 더빙.. 진짜 짜증나네요 목소리,짜증 나 목소리
1,3819312,흠...포스터보고 초딩영화줄....오버연기조차 가볍지 않구나,1,흠...포스터보고 초딩영화줄....오버연기조차 가볍지 않구나,포스터 초딩 영화 줄 오버 연기 가볍
2,10265843,너무재밓었다그래서보는것을추천한다,0,너무재밓었다그래서보는것을추천한다,
3,9045019,교도소 이야기구먼 ..솔직히 재미는 없다..평점 조정,0,교도소 이야기구먼 ..솔직히 재미는 없다..평점 조정,교도소 이야기 재미 없 평점 조정
4,6483659,사이몬페그의 익살스런 연기가 돋보였던 영화!스파이더맨에서 늙어보이기만 했던 커스틴 ...,1,사이몬페그의 익살스런 연기가 돋보였던 영화!스파이더맨에서 늙어보이기만 했던 커스틴 ...,사이몬페그 익살 연기 영화 스파이더맨 늙 보이 커스틴 던스트
...,...,...,...,...,...
149995,6222902,인간이 문제지.. 소는 뭔죄인가..,0,인간이 문제지.. 소는 뭔죄인가..,인간 문제 소 죄
149996,8549745,평점이 너무 낮아서...,1,평점이 너무 낮아서...,평점 낮
149997,9311800,이게 뭐요? 한국인은 거들먹거리고 필리핀 혼혈은 착하다?,0,이게 뭐요? 한국인은 거들먹거리고 필리핀 혼혈은 착하다?,한국인 거들먹거리 필리핀 혼혈 착하
149998,2376369,청춘 영화의 최고봉.방황과 우울했던 날들의 자화상,1,청춘 영화의 최고봉.방황과 우울했던 날들의 자화상,청춘 영화 최고봉 방황 우울 날 자화상


In [15]:
# tokenizing
from sklearn.feature_extraction.text import TfidfVectorizer
tfidfVectorizer = TfidfVectorizer(ngram_range=(1, 2))

In [16]:
tfidfVectorizer.fit(df_ratings['document_clean'])

TfidfVectorizer(ngram_range=(1, 2))

In [17]:
train_target = df_ratings['label']
train_feature = tfidfVectorizer.transform(df_ratings['document_clean'])

In [18]:
# train, test 나누기
from sklearn.model_selection import train_test_split

feature_train, feature_test, target_train, target_test = train_test_split(train_feature, train_target, test_size=0.2, random_state=42)

In [19]:
feature_train.shape, feature_test.shape, target_train.shape, target_test.shape

((120000, 342325), (30000, 342325), (120000,), (30000,))

In [20]:
from sklearn.ensemble import RandomForestClassifier
model = RandomForestClassifier(n_jobs=-1)

In [21]:
model.fit(feature_train, target_train)

RandomForestClassifier(n_jobs=-1)

In [22]:
model.predict(feature_test[120:125]), target_test[120:125]

(array([0, 1, 1, 0, 0]),
 51156     0
 129330    1
 67233     1
 115061    0
 17169     1
 Name: label, dtype: int64)

In [23]:
from sklearn.metrics import accuracy_score
target_train_predict = model.predict(feature_train)
accuracy_score(target_train, target_train_predict)

0.9307083333333334

In [24]:
from sklearn.metrics import accuracy_score
target_test_predict = model.predict(feature_test)
accuracy_score(target_test, target_test_predict)

0.7456

In [25]:
# classification_report로 model 성능 확인
from sklearn.metrics import classification_report
print(classification_report(target_test, target_test_predict))

              precision    recall  f1-score   support

           0       0.72      0.81      0.76     15164
           1       0.77      0.68      0.73     14836

    accuracy                           0.75     30000
   macro avg       0.75      0.74      0.74     30000
weighted avg       0.75      0.75      0.74     30000



In [26]:
text1 = "해리포터 생각하고 보면 조금 실망할 듯"

In [27]:
text1_mecab = mecab_tag([text1])
text1_mecab

['해리포터 생각 실망 듯']

In [28]:
model.predict(tfidfVectorizer.transform(text1_mecab))

array([0])

In [29]:
text2 = "가볍게 보기 좋은 영환데 개취 완전 저격- 호."

In [30]:
text2_mecab = mecab_tag([text2])
text2_mecab

['가볍 좋 개취 완전 저격']

In [31]:
model.predict(tfidfVectorizer.transform(text2_mecab))

array([1])